In [ ]:
!pip install transformers datasets pandas torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments


# Load the CSV file
df = pd.read_csv('datasets.csv')

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Split the dataset into training and validation sets
dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset as HFDataset
from sklearn.model_selection import train_test_split

# Load dataset
def load_data(csv_path):
    df = pd.read_csv(csv_path)
    df = df.dropna()
    train_texts, val_texts, train_summaries, val_summaries = train_test_split(
        df['text'].tolist(), df['summary'].tolist(), test_size=0.1
    )
    return train_texts, val_texts, train_summaries, val_summaries

# Convert to Hugging Face Dataset
def convert_to_hf_dataset(texts, summaries, tokenizer, max_length=512):
    encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=max_length)
    labels = tokenizer(summaries, truncation=True, padding='max_length', max_length=150)
    dataset = HFDataset.from_dict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': labels['input_ids']
    })
    return dataset

# Load model and tokenizer
def load_model():
    model_name = "t5-small"
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    return model, tokenizer, device

# Train model
def train_model(train_dataset, val_dataset, model, tokenizer):
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=5,
        save_steps=500,
        save_total_limit=2,
        logging_dir="./logs",
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )
    trainer.train()
    model.save_pretrained("./summarization_model")
    tokenizer.save_pretrained("./summarization_model")

# Summarization function
def summarize(text, model, tokenizer, device, max_length=150):
    input_text = "summarize: " + text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Move input tensors to the same device as the model
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length, num_beams=4, early_stopping=True)

    return tokenizer.decode(output[0], skip_special_tokens=True)

if __name__ == "__main__":
    csv_path = "/content/datasets.csv"  # Update with your file path
    train_texts, val_texts, train_summaries, val_summaries = load_data(csv_path)

    model, tokenizer, device = load_model()
    train_dataset = convert_to_hf_dataset(train_texts, train_summaries, tokenizer)
    val_dataset = convert_to_hf_dataset(val_texts, val_summaries, tokenizer)

    train_model(train_dataset, val_dataset, model, tokenizer)

    test_text = "The Hugging Face Transformers library provides an easy-to-use interface for working with state-of-the-art natural language processing models.It supports a wide range of tasks, including text classification, question answering, and text summarization.With just a few lines of code, you can fine-tune pre-trained models on your own dataset and deploy them for inference."
    summary = summarize(test_text, model, tokenizer, device)
    print("Generated Summary:", summary)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.503000,0.218897
2,0.267200,0.201220
3,0.240100,0.191538
4,0.221000,0.185720
5,0.221200,0.186989


Generated Summary: the Hugging Face Transformers library provides an easy-to-use interface for working with state-of-the-art natural language processing models. It supports a wide range of tasks, including text classification, question answering, and text summarization.


In [ ]:
model.save_pretrained("./summarization_model")
tokenizer.save_pretrained("./summarization_model")


('./summarization_model/tokenizer_config.json',
 './summarization_model/special_tokens_map.json',
 './summarization_model/spiece.model',
 './summarization_model/added_tokens.json')